# Libraries

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize
from scipy.interpolate import CubicSpline

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk


output_notebook()

Loading BokehJS ...

# Functions

In [2]:
# READS THE IMAGE AND CONVERTS IT TO RGBA IMG FOR DISPLAY #

def readImgAndConvertToRGBA(filename):

    # reads the image using simpleITK:
    img = sitk.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = sitk.GetArrayFromImage(img)
    
    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return array, rgba, sizex, sizey

In [3]:
# CONVERTS THE IMG TO RGBA FOR DISPLAY #

def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba

In [4]:
# READS ALL IMAGES IN THE PATH AND RETURNS THE MEDIAN IMAGE #

def medianImage(path, filename, nbOfImgs):

    imgs = []
    
    # reads the images and store the matrix in imgs:
    sizex=0 
    sizey =0
    for i in range(nbOfImgs):
        im = sitk.ReadImage(path+filename+str(i+1)+'.tif')
        sizex = im.GetWidth()
        sizey = im.GetHeight()
        ar = sitk.GetArrayFromImage(im)
        imgs.append(ar)

    print('Image size:', sizex, 'x', sizey, 'x 3 layers (R,G,B)')
    print('')

    # creation and filling of median image:
    medianImg = np.empty((sizey,sizex,3))
    for i in range(sizex):
        for j in range(sizey):
            for k in range(3):
                a = []
                for l in range(nbOfImgs):
                    a.append(imgs[l][j][i][k])
                medianImg[j][i][k] = np.median(a)
    
    return medianImg, sizex, sizey

In [5]:
# PRINT MEAN GREY VALUES:

def printGreyValues(rgbvalues):
    for i in range(len(rgbvalues)):
        print('Dose ', str(i), ':', rgbvalues[i][0], 'Gy') 
        print('  - red value:', rgbvalues[i][1])
        print('  - green value:', rgbvalues[i][2])
        print('  - blue value:', rgbvalues[i][3])
        print('')
    

# Reads the images

In [7]:
# READS THE IMAGE AND CALCULATES MEAN VALUES:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


# Inputs:

m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/09 - etalonnage lot 10151801 19juil2019/films a 24h/'
m_nbOfFiles = 5
m_filesName = "scan"

m_dimViewer = 600

dose = [799.7, 599.8, 399.9, 299.9, 199.9, 100.0, 50.4, 24.8]
#dose = [24.8, 50.4, 100.0, 199.9, 299.9, 399.9, 599.8, 799.7]

rectList = []
rectList.append([310,65,220,65])
rectList.append([310,162,220,65])
rectList.append([310,252,220,65])
rectList.append([310,337,220,65])
rectList.append([310,425,220,65])
rectList.append([310,510,220,65])
rectList.append([310,600,220,65])
rectList.append([310,690,220,65])

# Reads the images and computes the mean image:

#arrayimg, rgbaimg, sizex, sizey = readImgAndConvertToRGBA(m_path+m_filesName+'1'+'.tif')
medianImg, sizex, sizey = medianImage(m_path, m_filesName, m_nbOfFiles)
rgbaimg = convertToRGBA(medianImg, sizex, sizey)
arrayimg = medianImg


# Plots the areas of interest and calculates the mean rgb grey values in the areas:

p = figure(plot_width=int(m_dimViewer*0.95), plot_height=int(m_dimViewer*0.95*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Gafchromic image", toolbar_location="above")

p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

meanRGBvalues = []
for i in range(len(rectList)):
    p.rect(x=int(rectList[i][0]), y=rectList[i][1], width=rectList[i][2], 
                           height=rectList[i][3], fill_color="#0096ff", alpha=0.2)
    r = np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 0])
    g = np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 1])
    b = np.mean(arrayimg[int(rectList[i][1]-rectList[i][3]/2):int(rectList[i][1]+rectList[i][3]/2),
                           int(rectList[i][0]-rectList[i][2]/2):int(rectList[i][0]+rectList[i][2]/2), 2])
    meanRGBvalues.append([dose[i], r, g, b])

print("dONE !")

#printGreyValues(meanRGBvalues)
    

#show(p)

Image size: 593 x 738 x 3 layers (R,G,B)

dONE !


# RGB values over dose curves

In [8]:
# Plots RGB curves:

p1 = figure(plot_width=700, plot_height=400, title='RGB values', toolbar_location="above")
p2 = figure(plot_width=700, plot_height=400, title='Red/Blue Ratio', toolbar_location="above")

rvalues = []
gvalues = []
bvalues = []
rbvalues = []
for d, r, g, b in meanRGBvalues:
    rvalues.append(r)
    gvalues.append(g)
    bvalues.append(b)
    rbvalues.append(r/b)
    
p1.line(dose, rvalues, line_width=2, line_color='firebrick')
p1.line(dose, gvalues, line_width=2, line_color='green')
p1.line(dose, bvalues, line_width=2, line_color='blue')
p2.line(dose, rbvalues, line_width=2, line_color='darkorange')

show(column(p1,p2))

# R/B spline & polynomial fitting

In [9]:
# Comparaison fitting polynomial et spline R/B:

p1 = figure(plot_width=700, plot_height=400, title='RGB values to dose', toolbar_location="above")    
p1.line(rbvalues, dose, line_width=2, line_color='darkblue', legend='Data')

# fitting polynomial:
def func(x, a, b, c, d, e, f, g):
    return a*x**6 + b*x**5 + c*x**4 + d*x**3 + e*x**2 + f*x + g
params, params_covariance = optimize.curve_fit(func, rbvalues, dose)

# fitting spline: 
cs = CubicSpline(rbvalues, dose)

#Affichage des fits:
x = np.linspace(0.8, 1.6, num=91)
p1.line(x, func(x, params[0], params[1], params[2], params[3], params[4], params[5], params[6]), 
            line_width=2, line_color='darkorange', line_dash='dashed', legend='polynomial fit')

p1.line(x, cs(x), line_width=2, line_color='lightgreen', line_dash='dashed', legend='bspline fit')

p1.legend.location = "top_right"
p1.legend.click_policy="hide"

show(p1)

In [ ]:
# SAVE CALIBRATION DATA:

#  In the case of a bspline fitting:
file1 = open(m_path+"bSpline_data.txt","w+")

file1.write("BSpline fitting data \n\n")
file1.write("R/B value\tdose\n")
for i in range(len(dose)):
    file1.write(str(rbvalues[i])+'\t'+str(dose[i])+'\n')

file1.close()